In [7]:
# These are versions of some of our important functions
# that were duplicates or non-generalized forms that I
# am retiring to here in case something goes wrong

import numpy as np

In [8]:
def Margies_marginal_probabilities(ntwk_results): #a combined version of this with Ella's is kept in the real code
    #ntwk_results: dict, counts resulting from network run (should have 2^n entries)
    #marg_probs: array of length n, marginal probabilities that each qubit is 0,
        #from most significant to least significant
    
    n = len(list(ntwk_results.keys())[0])
    prob = np.zeros(n)
    total = sum(ntwk_results.values())

    for i in range(n):
        for key in ntwk_results:
            if int(key[i]) == 0:
                prob[i] += ntwk_results[key]
        prob[i] = prob[i]/total
    
    return prob

In [9]:
def Ella_3qubit_marginal_probabilities(pstates_dict):
    # Works for 3 qubits only
    C0, B0, A0, numtot = 0, 0, 0, 0

    for key in pstates_dict:
        Cstate=key[0]
        Bstate=key[1]
        Astate=key[2]
        numtot += pstates_dict[key]
        if Cstate == '0':
            C0 += pstates_dict[key]
        if Bstate == '0':
            B0 += pstates_dict[key]
        if Astate == '0':
            A0 += pstates_dict[key]
        
    ProbC0 = C0/numtot
    ProbB0 = B0/numtot
    ProbA0 = A0/numtot

    return ProbC0, ProbB0, ProbA0

In [10]:
def Ella_marginal_probabilities_general(pstates_dict):
    """Works for n qubits"""
    n = len(list(pstates_dict.keys())[0]) #number of qubits in state
    Probs = np.empty(n)
    numZeros, numtot = np.zeros(n), 0

    for key in pstates_dict:
        numtot += pstates_dict[key]
        for i in range(n):
            #print(i)
            state=key[i]
            if state == '0':
                numZeros[i] += pstates_dict[key]
    for i in range(n):
        Probs[i] = numZeros[i]/numtot

    return Probs

In [15]:
# Test the marginal prob functions with some simulated output from our 3 qubit run!

#pick the function to use (comment others out)
funct = Ella_marginal_probabilities_general
#funct = Ella_3qubit_marginal_probabilities
#funct = Margies_marginal_probabilities

result1 = {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}
print(funct(result1))

[0.73181152 0.62390137 0.65820312]
